# 任务一

In [ ]:
# 导入数据
import pandas as pd
df1 = pd.read_excel(r'D:\研究生资料\玛氏数据\5-22\chengdu_521.xlsx')
df_POI = pd.read_excel(r'D:\研究生资料\玛氏数据\chengdu_poi_category.xlsx')

## 一、求对口香糖影响较大的POI

1)	筛选出销售口香糖的门店，求出销售口香糖门店对应的POI；



In [ ]:
# 卖Chewing的'store_code'
Chewing_storeid=set(df1.loc[df1['ProductCategory']=='Chewing/Bubble Gum','store_code'])
Chewing_storeid=list(Chewing_storeid)
# 销售Chewing门店对应的POI
Chewing_POI=df_POI[df_POI['storecode'].isin(Chewing_storeid)]

2)	筛选出不销售口香糖的门店，求出不销售口香糖门店对应的POI；


In [ ]:
# 不卖Chewing的'store_code'
all_storeid = list(df1['store_code'])
NoChewing_storeid=list(set(all_storeid)-set(Chewing_storeid))
# 不销售Chewing门店对应的POI
NoChewing_POI=df_POI[df_POI['storecode'].isin(NoChewing_storeid)]

3)	筛选出销售口香糖平均下单订单时间大于6个月的门店，求出这些门店对应的POI；


In [ ]:
# 把Chewing的那部分分成下单周期小于6个月和下达周期大于6个月两类
Cstore_id_da6 = [] # 用来储存买Chewing的那部分下单周期大于6个月的'store_code'
Cstore_id_xiao6 = [] # 用来储存买Chewing的那部分下单周期小于等于6个月的'store_code'
for i in range(len(Chewing_storeid)):
    demo0 = df1.loc[df1['store_code']==Chewing_storeid[i]]
    # 这些门店可能同时买三种产品，拿出卖Chewing的记录
    demo0_1 = demo0.loc[demo0['ProductCategory']=='Chewing/Bubble Gum']
    # 选出订单时间
    demo1 = demo0_1['order_date']
    demo1 = demo1.sort_values()# 时间排序
    # 时间提取，不要小时。。。
    aa=demo1.map(lambda x: x.strftime('%Y-%m-%d'))
    # 将字符串转变为时间
    demo1=pd.to_datetime(aa)
    demo1_index = list(demo1.index)

    day=(demo1[demo1_index[-1]]-demo1[demo1_index[0]]).days
    
    #计算订单个数(将订单时间相等的订单当做一个订单)
    order_sum=list(set(list(demo1)))
    order_sum=len(order_sum)
    #interval = day/(order_sum-1)，要提前判断下分母是否等于0
    if order_sum-1==0:
        Cstore_id_da6.append(Chewing_storeid[i])
    else:
        interval = day/(order_sum-1)
        if interval>180:
            Cstore_id_da6.append(Chewing_storeid[i])
        else:
            Cstore_id_xiao6.append(Chewing_storeid[i]) 
            
            
# Cstore_id_da6门店对应的POI
Cstore_id_da6_POI=df_POI[df_POI['storecode'].isin(Cstore_id_da6)]

4)	用销售口香糖门店对应的POI减去口香糖平均下单时间大月6个月门店对应的POI，再减去不销售口香糖门店对应的POI，得到对口香糖影响较大的POI

In [ ]:
a1=set(Chewing_POI['s_cat_id'])
a2=set(Cstore_id_da6_POI['s_cat_id'])
a3=set(NoChewing_POI['s_cat_id'])
a4=list(a1-a2-a3)
df_aa = Chewing_POI[Chewing_POI['s_cat_id'].isin(a4)]
# 去除s_cat_value列的重复值
df_aa_1=df_aa.drop_duplicates(['s_cat_value'])
df_aa_1.to_excel(r'C:\Users\Administrator\Desktop\任务一（一）.xlsx')

## 二、口香糖在销量维度的最强POI

1)	求出各个门店中口香糖占总销量的比重；



In [ ]:
import numpy as np

# 卖Chewing的'store_code'
Chewing_storeid=set(df1.loc[df1['ProductCategory']=='Chewing/Bubble Gum','store_code'])
Chewing_storeid=list(Chewing_storeid)

keys_1 = []
values_1 = []
for i in range(len(Chewing_storeid)):
    demo1 = list(df1.loc[(df1['store_code']==Chewing_storeid[i])&(df1['ProductCategory']=='Chewing/Bubble Gum'),'case_ordered'])
    demo2 = list(df1.loc[(df1['store_code']==Chewing_storeid[i]),'case_ordered'])
    demo3 = np.array(demo1).sum()/np.array(demo2).sum()
    keys_1.append(Chewing_storeid[i])
    values_1.append(demo3)

2)	按照销量比重，将门店分为3类，比重小于30% （弱）、比重大于70%（强），比重在30%~70%之间（中）；


In [ ]:
data1 = {
    "store_code": keys_1,
    "prop":values_1
}
df_cc = pd.DataFrame(data1)
bins = [0, 0.3, 0.7, 1]
df_cc['Categories'] = pd.cut(df_cc['prop'], bins)
df_cc['four'] =df_cc['Categories'].cat.codes
# 求出每一类别对应的store_code
type_0_cc = list(df_cc.loc[df_cc['four']==0,'store_code'])
type_1_cc = list(df_cc.loc[df_cc['four']==1,'store_code'])
type_2_cc = list(df_cc.loc[df_cc['four']==2,'store_code'])

3)	求出（2）中每一类门店对应的POI;


In [ ]:
# 筛选出第一类门店对应的POI;
Type_0_cc = df_POI[df_POI['storecode'].isin(type_0_cc)]
Type_0_cc_1 = Type_0_cc.drop_duplicates(['s_cat_id'])
# Type_0_cc_1.to_excel(r'C:\Users\Administrator\Desktop\Chewing弱.xlsx')
# 筛选出第二类门店对应的POI;
Type_1_cc = df_POI[df_POI['storecode'].isin(type_1_cc)]
Type_1_cc_1 = Type_1_cc.drop_duplicates(['s_cat_id'])
# Type_1_cc_1.to_excel(r'C:\Users\Administrator\Desktop\Chewing中.xlsx')
# 筛选出第类三门店对应的POI;
Type_2_cc = df_POI[df_POI['storecode'].isin(type_2_cc)]
Type_2_cc_1 = Type_2_cc.drop_duplicates(['s_cat_id'])
# Type_1_cc_1.to_excel(r'C:\Users\Administrator\Desktop\Chewing强.xlsx')

4)	用强标签减去中标签再减去弱标签，求出对口香糖销量影响最大的POI（强标签独有集合）；

In [ ]:
b1=set(Type_0_cc_1['s_cat_id'])
b2=set(Type_1_cc_1['s_cat_id'])
b3=set(Type_2_cc_1['s_cat_id'])
b4=list(b3-b2-b1)
df_bb = Type_2_cc_1[Type_2_cc_1['s_cat_id'].isin(b4)]
df_bb.to_excel(r'C:\Users\Administrator\Desktop\任务一（二）.xlsx')

## 三、（一）和（二）的交集

1)	用“（一）对口香糖你影响较大的POI集合”和“（二）对口香糖销量影响最大的POI集合”求交集；


In [ ]:
c1 = list(set(a4)&set(b4))
df_cc = Type_2_cc_1[Type_2_cc_1['s_cat_id'].isin(c1)]
df_cc.to_excel(r'C:\Users\Administrator\Desktop\任务一（三1）.xlsx')

2)	强标签独有集合减去平均下单时间大于6个月门店对应的POI集合


In [ ]:
 强标签POI集合 Type_2_cc_1
# 平均下单时间大于6个月门店对应的POI集合 Cstore_id_da6_POI
d1=set(Type_2_cc_1['s_cat_id'])
d2=set(Cstore_id_da6_POI['s_cat_id'])
d3=list(d1-d2)
df_dd = Type_2_cc_1[Type_2_cc_1['s_cat_id'].isin(d3)]
df_dd.to_excel(r'C:\Users\Administrator\Desktop\任务二（三2）.xlsx')

3)	（2）中的POI集合与不销售口香糖门店对应的POI求交集


In [ ]:
# 不销售Chewing门店对应的POI NoChewing_POI
e1=set(NoChewing_POI['s_cat_id'])
e2=list(set(d3)&e1)
df_ee = Type_2_cc_1[Type_2_cc_1['s_cat_id'].isin(e2)]
df_ee.to_excel(r'C:\Users\Administrator\Desktop\任务一（三3）.xlsx')